In [1]:
import os
import numpy as np
import requests
import argparse
import json
import time
import logging
import csv

from multiprocessing import Pool, Process, Value, Lock

from requests.exceptions import ConnectionError, ReadTimeout, TooManyRedirects, MissingSchema, InvalidURL

In [2]:
# args
data_root = './data'
user_class_list = False
class_list = []
scrape_only_flickr = True # default True
number_of_classes = 1100
images_per_class = 10
use_class_list = False
debug = False
multiprocessing_workers = 8

In [3]:
if debug:
    logging.basicConfig(filename='imagenet_scarper.log', level=logging.DEBUG)

In [4]:
IMAGENET_API_WNID_TO_URLS = lambda wnid: f'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid={wnid}'

current_folder = os.path.abspath('.')

class_info_json_filename = 'imagenet_class_info.json'
class_info_json_filepath = os.path.join(current_folder, class_info_json_filename)

class_info_dict = dict()

In [5]:
with open(class_info_json_filepath) as class_info_json_f:
    class_info_dict = json.load(class_info_json_f)

In [6]:
classes_to_scrape = []

if use_class_list == True:
    for item in class_list:
        classes_to_scrape.append(item)
        if item not in class_info_dict:
            logging.error(f'Class {item} not found in ImageNete')
            exit()
            
elif use_class_list == False:
    potential_class_pool = []
    for key, val in class_info_dict.items():

        if scrape_only_flickr:
            if int(val['flickr_img_url_count']) * 0.9 > images_per_class:
                potential_class_pool.append(key)
        else:
            if int(val['img_url_count']) * 0.8 > images_per_class:
                potential_class_pool.append(key)

    if (len(potential_class_pool) < number_of_classes):
        logging.error(f"With {images_per_class} images per class there are {len(potential_class_pool)} to choose from.")
        logging.error(f"Decrease number of classes or decrease images per class.")
        exit()

    picked_classes_idxes = np.random.choice(len(potential_class_pool), number_of_classes, replace = False)

    for idx in picked_classes_idxes:
        classes_to_scrape.append(potential_class_pool[idx])
            

In [7]:
print("Picked the following clases:")
print([ class_info_dict[class_wnid]['class_name'] for class_wnid in classes_to_scrape ])

Picked the following clases:
['catmint', 'easy chair', 'ski rack', 'common cotton grass', 'Dane', 'marine mussel', 'locoweed', 'hall', 'armor', 'chador', 'guest of honor', 'bonefish', 'rowel', 'cowpea', 'parfait', 'bouillon', 'flying jib', 'White Russian', 'astrolabe', 'glow lamp', 'Melkite', 'hydrometer', 'marsh plant', 'oyster mushroom', 'maple', 'entrepreneur', 'metasequoia', 'level', 'foreground', 'linseed', 'rig', 'engine', 'shunt', 'Easter egg', 'coatdress', 'American wistaria', 'Mexican sunflower', 'pavilion', 'imperialist', 'centrist', 'Minuteman', 'sequencer', 'bread dough', 'alehouse', 'cock of the rock', 'asp', 'gasmask', 'bass guitar', 'poppy', 'panther', 'ridge', 'good-king-henry', 'sculptor', 'wild boar', 'boxcar', 'bitewing', 'buckram', 'valerian', 'television antenna', 'climbing frame', 'cornetfish', 'helminth', 'Tartuffe', 'header', 'push button', 'kneeler', 'impala lily', 'plum sauce', 'drinking vessel', 'common fennel', 'green alder', 'espionage agent', 'autopilot', 

In [10]:
imagenet_images_folder = os.path.join(data_root, 'imagenet_images')
if not os.path.isdir(imagenet_images_folder):
    os.mkdir(imagenet_images_folder)

In [11]:
scraping_stats = dict(
    all=dict(
        tried=0,
        success=0,
        time_spent=0,
    ),
    is_flickr=dict(
        tried=0,
        success=0,
        time_spent=0,
    ),
    not_flickr=dict(
        tried=0,
        success=0,
        time_spent=0,
    )
)

In [12]:
def add_debug_csv_row(row):
    with open('stats.csv', "a") as csv_f:
        csv_writer = csv.writer(csv_f, delimiter=",")
        csv_writer.writerow(row)

class MultiStats():
    def __init__(self):

        self.lock = Lock()

        self.stats = dict(
            all=dict(
                tried=Value('d', 0),
                success=Value('d',0),
                time_spent=Value('d',0),
            ),
            is_flickr=dict(
                tried=Value('d', 0),
                success=Value('d',0),
                time_spent=Value('d',0),
            ),
            not_flickr=dict(
                tried=Value('d', 0),
                success=Value('d', 0),
                time_spent=Value('d', 0),
            )
        )
    def inc(self, cls, stat, val):
        with self.lock:
            self.stats[cls][stat].value += val

    def get(self, cls, stat):
        with self.lock:
            ret = self.stats[cls][stat].value
        return ret

multi_stats = MultiStats()
    
if debug:
    row = [
        "all_tried",
        "all_success",
        "all_time_spent",
        "is_flickr_tried",
        "is_flickr_success",
        "is_flickr_time_spent",
        "not_flickr_tried",
        "not_flickr_success",
        "not_flickr_time_spent"
    ]
    add_debug_csv_row(row)
    
def add_stats_to_debug_csv():
    row = [
        multi_stats.get('all', 'tried'),
        multi_stats.get('all', 'success'),
        multi_stats.get('all', 'time_spent'),
        multi_stats.get('is_flickr', 'tried'),
        multi_stats.get('is_flickr', 'success'),
        multi_stats.get('is_flickr', 'time_spent'),
        multi_stats.get('not_flickr', 'tried'),
        multi_stats.get('not_flickr', 'success'),
        multi_stats.get('not_flickr', 'time_spent'),
    ]
    add_debug_csv_row(row)

def print_stats(cls, print_func):

    actual_all_time_spent = time.time() - scraping_t_start.value
    processes_all_time_spent = multi_stats.get('all', 'time_spent')

    if processes_all_time_spent == 0:
        actual_processes_ratio = 1.0
    else:
        actual_processes_ratio = actual_all_time_spent / processes_all_time_spent

    #print(f"actual all time: {actual_all_time_spent} proc all time {processes_all_time_spent}")

    print_func(f'STATS For class {cls}:')
    print_func(f' tried {multi_stats.get(cls, "tried")} urls with'
               f' {multi_stats.get(cls, "success")} successes')

    if multi_stats.get(cls, "tried") > 0:
        print_func(f'{100.0 * multi_stats.get(cls, "success")/multi_stats.get(cls, "tried")}% success rate for {cls} urls ')
    if multi_stats.get(cls, "success") > 0:
        print_func(f'{multi_stats.get(cls,"time_spent") * actual_processes_ratio / multi_stats.get(cls,"success")} seconds spent per {cls} succesful image download')



lock = Lock()
url_tries = Value('d', 0)
scraping_t_start = Value('d', time.time())
class_folder = ''
class_images = Value('d', 0)

In [15]:
def get_image(img_url):

    #print(f'Processing {img_url}')

    #time.sleep(3)

    if len(img_url) <= 1:
        return


    cls_imgs = 0
    with lock:
        cls_imgs = class_images.value

    if cls_imgs >= images_per_class:
        return

    logging.debug(img_url)

    cls = ''

    if 'flickr' in img_url:
        cls = 'is_flickr'
    else:
        cls = 'not_flickr'
        if scrape_only_flickr:
            return

    t_start = time.time()

    def finish(status):
        t_spent = time.time() - t_start
        multi_stats.inc(cls, 'time_spent', t_spent)
        multi_stats.inc('all', 'time_spent', t_spent)

        multi_stats.inc(cls,'tried', 1)
        multi_stats.inc('all', 'tried', 1)

        if status == 'success':
            multi_stats.inc(cls,'success', 1)
            multi_stats.inc('all', 'success', 1)

        elif status == 'failure':
            pass
        else:
            logging.error(f'No such status {status}!!')
            exit()
        return


    with lock:
        url_tries.value += 1
        if url_tries.value % 250 == 0:
            print(f'\nScraping stats:')
            print_stats('is_flickr', print)
            print_stats('not_flickr', print)
            print_stats('all', print)
            if debug:
                add_stats_to_debug_csv()

    try:
        img_resp = requests.get(img_url, timeout = 1)
    except ConnectionError:
        logging.debug(f"Connection Error for url {img_url}")
        return finish('failure')
    except ReadTimeout:
        logging.debug(f"Read Timeout for url {img_url}")
        return finish('failure')
    except TooManyRedirects:
        logging.debug(f"Too many redirects {img_url}")
        return finish('failure')
    except MissingSchema:
        return finish('failure')
    except InvalidURL:
        return finish('failure')

    if not 'content-type' in img_resp.headers:
        return finish('failure')

    if not 'image' in img_resp.headers['content-type']:
        logging.debug("Not an image")
        return finish('failure')

    if (len(img_resp.content) < 1000):
        return finish('failure')

    logging.debug(img_resp.headers['content-type'])
    logging.debug(f'image size {len(img_resp.content)}')

    img_name = img_url.split('/')[-1]
    img_name = img_name.split("?")[0]

    if (len(img_name) <= 1):
        return finish('failure')

    img_file_path = os.path.join(class_folder, img_name)
    logging.debug(f'Saving image in {img_file_path}')

    with open(img_file_path, 'wb') as img_f:
        img_f.write(img_resp.content)

        with lock:
            class_images.value += 1

        logging.debug(f'Scraping stats')
        print_stats('is_flickr', logging.debug)
        print_stats('not_flickr', logging.debug)
        print_stats('all', logging.debug)

        return finish('success')

from tqdm import tqdm
    
for class_wnid in tqdm(classes_to_scrape):

    class_name = class_info_dict[class_wnid]["class_name"]
    print(f'Scraping images for class \"{class_name}\"')
    url_urls = IMAGENET_API_WNID_TO_URLS(class_wnid)
    
    class_folder = os.path.join(imagenet_images_folder, class_name)
    if not os.path.exists(class_folder):
        os.mkdir(class_folder)
    else:
         continue
    
    time.sleep(0.05)
    try :
        resp = requests.get(url_urls)
    except ConnectionError:
        continue
    
    
    class_images.value = 0

    urls = [url.decode('utf-8') for url in resp.content.splitlines()]

    #for url in  urls:
    #    get_image(url)

    print(f"Multiprocessing workers: {multiprocessing_workers}")
    with Pool(processes=multiprocessing_workers) as p:
        p.map(get_image,urls)



  0%|          | 0/1100 [00:00<?, ?it/s]

Scraping images for class "catmint"
Scraping images for class "easy chair"
Multiprocessing workers: 8



  0%|          | 2/1100 [00:16<2:27:24,  8.05s/it]

Scraping images for class "ski rack"
Multiprocessing workers: 8



  0%|          | 3/1100 [00:24<2:28:33,  8.13s/it]

...

Scraping images for class "thrips"
Multiprocessing workers: 8



 18%|█▊        | 203/1100 [1:08:45<2:26:13,  9.78s/it]

Scraping images for class "lyme grass"
Multiprocessing workers: 8



100%|██████████| 1100/1100 [1:08:57<00:00,  3.76s/it] 

Scraping images for class "milk float"
Scraping images for class "fire tongs"
Scraping images for class "chamois"
Scraping images for class "row house"
Scraping images for class "banded krait"
Scraping images for class "hand mower"
Scraping images for class "gopher"
Scraping images for class "convolvulus"
Scraping images for class "clarified butter"
Scraping images for class "Coigue"
Scraping images for class "lupine"
Scraping images for class "flowerbed"
Scraping images for class "Mountie"
Scraping images for class "hot sauce"
Scraping images for class "solar dish"
Scraping images for class "great grandchild"
Scraping images for class "Scandinavian"
Scraping images for class "ticking"
Scraping images for class "dishtowel"
Scraping images for class "palomino"
Scraping images for class "toasting fork"
Scraping images for class "water gauge"
Scraping images for class "garland flower"
Scraping images for class "Wilson's snipe"
Scraping images for class "European swift"
Scraping images for